In [1]:
# import scripts.control_utils as cu
from importlib import reload
import matplotlib.pyplot as plt
from annotator.midas import MidasDetector
from annotator.canny import CannyDetector
from ldm.data.util import resize_image_pil
import torch
import torchvision.transforms as tt
import numpy as np

from PIL import Image
# reload(cu)
from safetensors.torch import save_file
import glob 

torch.cuda.set_device('cuda:2')



In [2]:
import scripts.control_utils as cu
reload(cu)
import os

def load_model_from_dic (path_dic, Lora_kitti, Lora_cubes):

    config_path_depth = path_dic['yaml']
    model = cu.create_model(config_path_depth)

    ckpt = torch.load(path_dic['ckpt'], map_location='cpu')
    model.load_state_dict(ckpt['state_dict'], strict=False)

    if Lora_kitti: 
        LORA_ckpt = torch.load('/export/data/vislearn/rother_subgroup/sd15_kitti.ckpt', map_location='cpu')
        model.load_state_dict(LORA_ckpt['state_dict'], strict=False)
    if Lora_cubes: 
        LORA_ckpt = torch.load('/export/data/vislearn/rother_subgroup/sd15_simplecubes.ckpt', map_location='cpu')
        model.load_state_dict(LORA_ckpt['state_dict'], strict=False)
    
    return model.to('cuda')

def get_paths (root, yaml_path, ckpt_path):
    path_dic = {}
    path_dic['yaml'] = os.path.join(root, yaml_path)
    path_dic['ckpt'] = os.path.join(root, ckpt_path)
    return path_dic

model_paths = {}
model_paths['CNXS_0.2'] = get_paths(root='/export/data/vislearn/rother_subgroup/dzavadsk/models/training_models/v15/2023-10-28T00-52-54_v15_03p_depth_add_bs24/',
                                    yaml_path = 'configs/2023-10-28T00-52-54-project.yaml',
                                    ckpt_path = 'checkpoints/trainstep_checkpoints/45000.ckpt'
                                    )
model_paths['CNXS_1.4'] = get_paths(root='/export/data/vislearn/rother_subgroup/dzavadsk/models/training_models/v15/2023-10-28T00-49-53_v15_03p_depth_cat_bs24/',
                                    yaml_path = 'configs/2023-10-28T00-49-53-project.yaml',
                                    ckpt_path = 'checkpoints/trainstep_checkpoints/45000.ckpt'
                                    )
model_paths['CNXS_20'] = get_paths(root='/export/data/vislearn/rother_subgroup/dzavadsk/models/trained_models/ControlNet/sd/v15/midas/2023-10-31T11-53-42_cross_encD_cat_20p_depth_bs16/',
                                    yaml_path = 'configs/project.yaml',
                                    ckpt_path = 'checkpoints/trainstep_checkpoints/45000.ckpt'
                                    )


/export/home/ffeiden/miniconda3/envs/.pt13/lib/python3.8/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


# SD 2.1

In [8]:
image_root = '/export/data/ffeiden/PaperControlnetXS/512_images/'
size = 512
reload(glob)

paths = glob.glob(image_root+'*_depth*.png')

instances = []
for im_path in paths: 
    images = {}
    name = im_path.split('/')[-1]
    name, alternative = name.split('_depth')[0], name.split('_depth')[1]
    if '_alt' in alternative: 
        alternative = True
    else: 
        alternative = False
    
    image = cu.get_image(image_root + name + '.png')
    depth = Image.open(im_path)
    if not depth.mode == 'L':
        depth = depth.convert("L") 
    images['original'] = image
    images['depth'] = np.array(depth).astype(np.float32)/255.
    images['Prompt'] = name
    images['alt'] = alternative
    # print(np.min(images['depth']), np.max(images['depth']), images['depth'].shape)
    instances.append(images)


cube = instances[0]
print(np.max(cube['depth']), np.min(cube['depth']))

midas_depth = cu.get_midas_depth(cube['original'])
midas_depth = midas_depth.astype(np.float32) / 255.
print(midas_depth.max(), midas_depth.min())



1.0 0.18431373
1.0 0.0


In [4]:
### Automate sampling ###
def sample (model, net, subfolder, root):
    
    os.mkdir(root+ net + subfolder)
    os.mkdir(root + net + subfolder + 'AA_grids')
    
    for current in instances: 
        for seed in [1995, 1996, 1998, 2024]:

            if current['alt']:
                sampe_name = current['Prompt'] + '_alt'
            else: 
                sampe_name = current['Prompt']

            samples, controls = cu.get_sd_sample(
                guidance=current['depth'],
                ddim_steps=50,
                num_samples=12,
                model=model,
                shape=[4, size // 8, size // 8],
                control_scale=1., # currently not working
                prompt=current['Prompt'],
                seed=seed,
                n_prompt='',
            )

            to_save = Image.fromarray(cu.create_image_grid(samples))
            to_save.save(root+net+subfolder+'AA_grids/'+sampe_name+'_seed'+str(seed)+'.png')

            # Generate score 
            best_depth_score = 999.999
            best_gen = None
            best_org_depth = None
            best_gen_depth = None

            for i in range(samples.shape[0]):
                gen_img = samples[i]
                orig_img = current['original']
                
                orig_depth = current['depth']
                gen_depth = cu.get_midas_depth(gen_img)

                orig_depth = np.array(orig_depth).astype(np.float32) / 255.
        
                gen_depth = gen_depth.astype(np.float32) / 255.
                
                to_save = Image.fromarray(samples[i])
                to_save.save(root+net+subfolder+sampe_name+'_seed'+str(seed)+'_samp'+str(i)+'.png')
                


In [5]:
root = '/export/data/ffeiden/ResultsControlNetXS/'
net = 'CNXS/'

for name_net in model_paths: 
    
    model = load_model_from_dic(model_paths[name_net], False, False)
    subfolder = name_net + '_Normal/'
    sample(model, net, subfolder, root)

    model = load_model_from_dic(model_paths[name_net], True, False)
    subfolder = name_net + '_kitti/'
    sample(model, net, subfolder, root)

    model = load_model_from_dic(model_paths[name_net], False, True)
    subfolder = name_net + '_cubes/'
    sample(model, net, subfolder, root)

TwoStreamControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280,

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.layer_norm2.weight', 'vision_model.encoder.layers.8.self_attn.q_proj.weight', 'vision_model.encoder.layers.16.mlp.fc1.weight', 'vision_model.encoder.layers.7.self_attn.q_proj.weight', 'vision_model.encoder.layers.13.self_attn.out_proj.weight', 'vision_model.encoder.layers.9.layer_norm2.weight', 'vision_model.encoder.layers.4.layer_norm2.bias', 'vision_model.encoder.layers.9.self_attn.v_proj.bias', 'vision_model.embeddings.position_embedding.weight', 'vision_model.encoder.layers.18.layer_norm1.weight', 'vision_model.encoder.layers.3.self_attn.v_proj.weight', 'vision_model.encoder.layers.5.self_attn.k_proj.bias', 'vision_model.encoder.layers.16.self_attn.out_proj.bias', 'vision_model.encoder.layers.19.self_attn.q_proj.weight', 'vision_model.encoder.layers.23.layer_norm2.bias', 'vision_model.encoder.layers.9.layer_norm2.bias', 'vision_model.

Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is 768 and using 8 heads.
Setting up Me

/export/home/ffeiden/miniconda3/envs/.pt13/lib/python3.8/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


Loaded model config from [/export/data/vislearn/rother_subgroup/dzavadsk/models/training_models/v15/2023-10-28T00-52-54_v15_03p_depth_add_bs24/configs/2023-10-28T00-52-54-project.yaml]


...

DDIM Sampler: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]
Global seed set to 1998


Data shape for DDIM sampling is (12, 4, 64, 64), eta 0.5
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]
Global seed set to 2024


Data shape for DDIM sampling is (12, 4, 64, 64), eta 0.5
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]
